## Data loading

In [1]:
# Load data and packages
import csv
from collections import Counter
from datetime import datetime, timedelta
import folium
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import math
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pmdarima as pm
from pmdarima import auto_arima
from scipy.stats import boxcox
from scipy.special import inv_boxcox
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA


directory_path = os.getcwd() + "\\Data\\sorting_event_volumes_2023.csv"

df = pd.read_csv(directory_path)

## Data cleaning

In [2]:
def fill_missing_events(df):
    df_filled_list = []
    
    for center in df['sorting_center_name'].unique():
        df_center = df[df['sorting_center_name'] == center]
        output_belts = df_center['output_belt'].unique()
        
        min_date = df_center['scanning_date'].min()
        max_date = df_center['scanning_date'].max()

        all_dates = pd.date_range(start=pd.Timestamp(year=min_date.year, month=1, day=1), end=pd.Timestamp(year=max_date.year, month=max_date.month, day=1) + pd.offsets.MonthEnd(0))
        
        all_combinations = pd.MultiIndex.from_product(
            [[center], all_dates, output_belts],
            names=['sorting_center_name', 'scanning_date', 'output_belt']
        )
        
        all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()
        df_filled_center = pd.merge(all_combinations_df, df_center, 
                                    on=['sorting_center_name', 'scanning_date', 'output_belt'], 
                                    how='left')
        
        df_filled_center['no_of_events'] = df_filled_center['no_of_events'].fillna(0.0001)
        df_filled_list.append(df_filled_center)
    
    df_filled = pd.concat(df_filled_list, ignore_index=True)
    
    return df_filled

## Data preparation

In [3]:
def prepare_data(df):
    # Data cleaning
    print("Number of rows original dataset is: " + str(df.shape[0]))

    df = df.loc[df["event_type"] == "LAJ", :]
    df.drop(['event_location', 'input_belt', 'position'], axis=1, inplace = True)
    df.dropna(inplace = True)
    df['output_belt'] = df['output_belt'].astype(int)
    df = df.groupby(['sorting_center_name', 'scanning_date', 'output_belt'], as_index = False)['no_of_events'].sum()
    df['scanning_date'] = pd.to_datetime(df['scanning_date'])

    # We add 0 events for all dates without orders 
    df = fill_missing_events(df)

    print("Number of rows cleaned dataset is: " + str(df.shape[0]))

    # Data preparation
    df['day'] = df['scanning_date'].dt.day
    df['month'] = df['scanning_date'].dt.month
    df['weekday'] = df['scanning_date'].dt.dayofweek + 1
    df['week'] = df['scanning_date'].dt.isocalendar().week
    df['week_of_month'] = (df['day'] - 1) // 7 + 1
    df['yearday'] = df['scanning_date'].dt.day_of_year
    df['weekday_sin'] = np.sin(df['weekday'] / 7 * 2 * np.pi)
    df['weekday_cos'] = np.cos(df['weekday'] / 7 * 2 * np.pi)

    return df

In [ ]:
df = prepare_data(df)

## Descriptive statistics

### Correlation matrix
We construct a correlation matrix to investigate which features correlate most with the number of events. This could give an indication with features would add most to prediction the number of events in the future. We observe that the correlation with number of events the day before is the strongest. Next the correlation is the strongest with output belt, weekday and yearday cosine.

In [ ]:
# Correlation matrix

# Add no_of_events previous day
df_prev_yearday = df.copy()
df_prev_yearday['yearday_yesterday'] = df_prev_yearday['yearday'] + 1  # Adjusting to match yearday - 1

df_correlations = pd.merge(
    df, 
    df_prev_yearday[['sorting_center_name', 'output_belt', 'yearday_yesterday', 'no_of_events']],  # only keep necessary columns from right
    left_on=['sorting_center_name', 'output_belt', 'yearday'],  # match on the current 'yearday' from the left df
    right_on=['sorting_center_name', 'output_belt', 'yearday_yesterday'],  # match on the 'yearday_yesterday' from the right df
    how='left',  # Perform a left join to keep all rows from the left dataframe
    suffixes=('', '_yesterday')
)

df_correlations = df_correlations.drop(columns = ["yearday_yesterday"])

# Create correlation matrix
target_covariance_matrix = df_correlations.drop(columns = ["sorting_center_name"]).corr().round(2)
#target_covariance_matrix_events = df_correlations.drop(columns = ["sorting_center_name"]).corr()[["no_of_events"]]
sns.heatmap(target_covariance_matrix,annot=True, cmap='coolwarm')

In [7]:
# For analysis exclude the null events
filtered_df = df[df['no_of_events'] >= 1]

In [ ]:
event_counts = filtered_df['no_of_events'].value_counts().sort_index()

plt.figure(figsize=(10, 6))
plt.bar(event_counts.index, event_counts.values)
plt.xscale('log')
plt.xlabel('Number of Events (log-scale)')
plt.ylabel('Frequency')
plt.title('Frequency of Number of Events')
plt.show()

### Sorting centers
We have six sorting centers located in Finland. The Vantaa sorting center is situated near the capital, Helsinki, and adjacent to the Helsinki international airport. In contrast, the other centers are located in smaller, more rural cities. A violin plot depicting the distribution of total daily events at each sorting center reveals significantly higher volumes at Vantaa, while the other centers show comparable levels. This prompts further investigation into the similarities among these sorting centers, specifically regarding the potential for predicting trends between urban and rural areas.

In [ ]:
locations = {}
locations["KUOPIO"] = (62.894785, 27.666586)
locations["LIETO"] = (60.503899,22.4322587)
locations["OULU"] = (65.007693, 25.471332)
locations["SEINÄJOKI"] = (62.7902205,22.7469712)
locations["TAMPERE"] = (61.499625, 23.751267)
locations["VANTAA"] = (60.294196, 25.035834)

avg_lat = sum(lat for lat, _ in locations.values()) / len(locations)
avg_lng = sum(lng for _, lng in locations.values()) / len(locations)
m = folium.Map(location=[avg_lat, avg_lng], zoom_start=6, tiles='CartoDB positron')

# Step 1: Add markers for each sorting center and track bounds
bounds = []

# Step 2: Add markers for each sorting center
for name, (lat, lng) in locations.items():
    folium.Marker(
        location=[lat, lng],
        icon=folium.DivIcon(
            html='<div style="background-color: #C50000; width: 12px; height: 12px; border: none;"></div>'
        ),
        popup=name  # Display the sorting center name on click
    ).add_to(m)
    # Add the current location to bounds
    bounds.append((lat, lng))

# Step 2: Fit the map bounds to show all markers
m.fit_bounds(bounds)

# Display the map
m

In [10]:
def plot_violin(df, period = "yearday"):
    if period == "yearday":
        df_period = df.groupby(["yearday"])["no_of_events"].sum().reset_index()
    else:
        df_period = df.groupby(["yearday", period])["no_of_events"].sum().reset_index()

    plt.figure(figsize=(10, 6))
    if period == "yearday":
        sns.violinplot(y="no_of_events", data=df_period)
    else:
        sns.violinplot(x = period, y="no_of_events", data=df_period)

    if period == "sorting_center_name":
        period = "sorting center"
    plt.xlabel("{}".format(period).capitalize())
    plt.ylabel("Number of Events")
    plt.title("Violin Plot of Number of Events by {}".format(period).capitalize())

    plt.xticks(rotation=90)
    plt.show()

In [ ]:
# Yearly per sorting center
plot_violin(df, "sorting_center_name")

### Outlier analysis
The violin plots indicate that all sorting centers have extreme outliers in the number of events compared to their mean. In the following analysis, we will further explore the variability of the number of events across these centers.

In [ ]:
fig = px.strip(filtered_df, 
                x='no_of_events', 
                y='sorting_center_name', 
                color='sorting_center_name',
                title='Spread of Number of Events by Sorting Center',
                labels={'no_of_events': 'Number of Events', 'sorting_center_name': 'Sorting Center'},
                stripmode='overlay')

fig.show()

We observed that most outliers occur at VANTAA. Specifically, there are 284 instances where the number of events exceeds 5000 and 234 instances where it surpasses 10,000. However, no particular day stands out, as the most frequent day occurs only 4 times. Interestingly, output belt 109 accounts for 254 days with over 5000 events and 233 days with over 10,000 events. This means only one other instance of 10,000+ events happens on a different belt, suggesting that the identified outliers reflect consistently high event volumes on belt 109 rather than sporadic spikes.

In [ ]:
# Outlier analysis
df_VANTAA_oultiers = df[(df["sorting_center_name"] == "VANTAA") & (df["no_of_events"] > 5000)]
print(df_VANTAA_oultiers.shape[0])

df_VANTAA_oultiers["yearday"].value_counts()
df_VANTAA_oultiers["output_belt"].value_counts()


## Data visualizations

### Aggregate demand planning horizon

We will focus on the sorting center VANTAA when creating data visualizations

In [14]:
# Sorting center is VANTAA
df_VANTAA = df[df["sorting_center_name"] == "VANTAA"]

In [15]:
# Aggregate demand planning horizon (Mats)
def total_orders_per_period(df, period):
    totals = df.groupby(period)['no_of_events'].sum()

    timeframe = ""

    if period == "day":
        timeframe = "month"
        
        valid_months = {day: 12 for day in range(1, 29)}
        valid_months[29] = 11
        valid_months[30] = 11
        valid_months[31] = 7

        year = df.iloc[0]["scanning_date"].year
        if (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)):
            valid_months[29] = 12

        for index in range(len(totals)):
            totals[index+1] = totals[index+1] / valid_months[index+1]
    elif period == "weekday":
        timeframe = "week"
    elif period == "week":
        timeframe = "year"

    plt.figure(figsize=(20, 6))
    plt.plot(totals.index, totals.values, marker='o')
    plt.xlabel('{}'.format(period).capitalize())
    plt.ylabel('Number of events')
    plt.ylim(0, 1.2 * totals.max())
    plt.title('Number of events in a {} over the {}'.format(period, timeframe))
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()

### Demand over the year

Below we show the total number of events for each week over the planning horizon of a year. We observe that the demand is relatively stable over the year except of a sharp increase in the last weeks of the year. This is probably due to Black Friday at the end of November and Christmas in December. Furthermore we observe that there are 5 days for which all sorting centers are closed. These are April 9 (Easter Sunday), June 24 (Midsummer Holiday), December 24 (Christmas Eve), December 25 (Christmas), December 31 (New Years').

In [ ]:
total_orders_per_period(df_VANTAA, "week")

### Boxplots

Below, we present violin plots showing demand distribution over a year, broken down by month and weekday. Overall, daily demand typically ranges between 50,000 and 160,000, with a high density around 150,000.

The monthly plots reveal some interesting patterns. December's average demand is lower than most months, likely due to the reduced activity between Christmas and New Year, which is also evident in the planning horizon plot. Notably, demand in November and December fluctuates significantly, as shown by the peaks in the violin plots, while March and August show a relatively narrow range.

Looking at the weekday patterns, demand during the workweek is noticeably higher than on weekends, with Sunday showing almost no demand. Tuesday consistently has the highest demand during the week, while Friday has the lowest. This weekday pattern suggests a cyclical trend, resembling a sine or cosine wave. Incorporating such patterns could maybe enhance the accuracy of demand forecasting.

In [ ]:
# Yearly
plot_violin(df_VANTAA)

# Monthly
plot_violin(df_VANTAA, "month")

# Weekday
plot_violin(df_VANTAA, "weekday")

### Day of the month

Below we present the demand distribution over a month by day of the month. We normalized the values since not all day of the month occur with the same frequency. We clearly see peaks at the 13th, 20th and 27th of the month. Interstingly those have a week interval in between, indicating that is maybe has to due with the fact that they occur more often at weekdays instead of weekends.

In [ ]:
total_orders_per_period(df_VANTAA, "day")

## Linear Regression Model
Before trying to estimate a linear regression, it might be interesting to study the correlations of the variables with the number of events.

In [ ]:

#determine correlations
target_covariance_matrix = adf.corr()[['no_of_events']]
print(target_covariance_matrix)
sns.heatmap(target_covariance_matrix,annot=True, cmap='coolwarm')

This colormap shows that the orders correlate with the day of the week. It was also attempted to put the day of the year in a trigonometric function with a period of 7 days, for which higher correlation was found with the cosine function. The reason is that the beginning of the week is more busy than the end, and the cosine function has its peak at 0.

For the linear regression model, lag data has to be created. Lags are the datapoints of x steps back in time. The best result was found by using 2 lags when estimating the model only on aggregate data of the large Vantaa sorting center.

In [ ]:
vantaa = adf[adf['sorting_center_name']=='VANTAA']
vantaa
# Create lag features (e.g., lag 1, lag 2)
def create_lag_features(series, lags):
    lagged_data = pd.DataFrame()
    for lag in range(1, lags+1):
        lagged_data[f'lag_{lag}'] = series.shift(lag)
    return lagged_data

# Create features and target
lags = 2  # You can change the number of lags
X = create_lag_features(vantaa['no_of_events'], lags)
X.dropna(inplace=True)
y = vantaa['no_of_events'][lags:]  # Target variable is the actual series shifted by the number of lags
dates = vantaa['scanning_date'][lags:]  # Corresponding dates for the target

# Split the data into train and test sets
X_train, X_test, y_train, y_test, dates_train, dates_test = train_test_split(
    X, y, dates, test_size=0.2, shuffle=False
)

# Create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Test RMSE: {rmse}")

In [ ]:
# Plot the actual vs predicted values
plt.figure(figsize=(10, 6))

# Plot actual values
plt.plot(dates_test, y_test, label='Actual Values', color='blue', marker='o')

# Plot predicted values
plt.plot(dates_test, y_pred, label='Predicted Values', color='red', linestyle='--', marker='x')

# Add labels and title
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Actual vs Predicted Values Over Time')
plt.xticks(rotation=45)
plt.legend()

# Show plot
plt.tight_layout()
plt.show()

## (Artificial) Neural Network

## ARIMA

### Hyperparameter tuning
We need to develop a model for each output belt at every sorting center. To ensure the accuracy of our models, we will train the hyperparameters p, d an q using data from the first nine months. These hyperparameters will be saved for future forecasts.

In the context of ARIMA:
- p represents the number of lag observations included in the model (the autoregressive term).
- d denotes the degree of differencing needed to make the time series stationary.
- q indicates the size of the moving average window, which reflects the number of lagged forecast errors in the prediction equation.

In [4]:
def tune_hyperparameters(df_output_belt):
    # Perform Box-Cox transformation
    #df_output_belt["no_of_events_boxcox"], lam = boxcox(df_output_belt["no_of_events"])
    #df_output_belt["no_of_events_diff"] = df_output_belt["no_of_events_boxcox"].diff()
    #df_output_belt.dropna(inplace=True)

    # Tune the model
    tuning_model = auto_arima(df_output_belt["no_of_events"], 
                               seasonal=False, 
                               stepwise=True,  
                               suppress_warnings=True, 
                               trace=False)

    
    p, d, q = tuning_model.order
    return (df_output_belt["output_belt"].iloc[0], p, d, q)

def train_ARIMA(df):
    sorting_center_names = df["sorting_center_name"].unique()

    for sorting_center_name in sorting_center_names:
        df_sorting_center = df[df["sorting_center_name"] == sorting_center_name]
        df_sorting_center.drop(["sorting_center_name"], axis=1, inplace=True)
        
        hyperparameterList = []
        output_belts = df_sorting_center["output_belt"].unique()

        results = Parallel(n_jobs=-1)(delayed(tune_hyperparameters)(
            df_sorting_center[df_sorting_center["output_belt"] == output_belt]) 
            for output_belt in output_belts)
        
        hyperparameterList.extend(results)

        hyperparameter_df = pd.DataFrame(hyperparameterList, columns=["output belt", "p", "d", "q"])
        hyperparameter_df.to_csv(f'Data/hyperparameters ARIMA/hyperparameters_ARIMA_{sorting_center_name}.csv', index=False)

In [ ]:
# Hyperparameter tuning
start = datetime.now()
train = df[df["month"] <= 9] # Train on first 9 months
train_ARIMA(train)
end = datetime.now()
print("Running time training ARIMA is: " + str(end-start))

In [6]:
def plot_forecast(starting_date, planning_horizon, train, forecasts, model, test = pd.DataFrame([])):
    fig = go.Figure()
    if test.shape[0] == 0:
        scanning_dates = pd.date_range(start=starting_date, periods=planning_horizon).strftime('%Y-%m-%d').tolist()
        fig.add_trace(go.Scatter(x=scanning_dates, y=forecasts, name="Forecast"))
    if test.shape[0] > 0:
        fig.add_trace(go.Scatter(x=train["scanning_date"], y=train["no_of_events"], name="Train"))
        fig.add_trace(go.Scatter(x=test["scanning_date"], y=test["no_of_events"], name="Test"))
        fig.add_trace(go.Scatter(x=test["scanning_date"], y=forecasts, name="Test"))
    fig.update_layout(template="simple_white", font=dict(size=18), title_text=model,
                      width=650, title_x=0.5, height=400, xaxis_title="scanning_date",
                      yaxis_title="no_of_events")
    
    return fig.show()

### Test ARIMA model

In [7]:
# Test ARIMA per output belt
def forecast_ARIMA(starting_date, planning_horizon, train_df, test_df = pd.DataFrame([])):
    sorting_center_names = df["sorting_center_name"].unique()

    forecast_dict = {}
    if test_df.shape[0] > 0:
        daily_errors = {}
        MSE_dict = {}
        VSE_dict = {}
        MAE_dict = {}

    for sorting_center_name in sorting_center_names:
        hyperparameters = pd.read_csv("Data/hyperparameters ARIMA/hyperparameters_ARIMA_{}.csv".format(sorting_center_name))

        train_sorting_center = train_df[train_df["sorting_center_name"] == sorting_center_name]

        if test_df.shape[0] > 0:
            test_sorting_center = test_df[test_df["sorting_center_name"] == sorting_center_name]

        output_belts = train_sorting_center['output_belt'].unique()

        for output_belt in output_belts:
            train_output_belt = train_sorting_center[train_sorting_center["output_belt"] == output_belt]

            if test_df.shape[0] == 0:
                test_output_belt = pd.Dataframe([])
            elif test_df.shape[0] > 0:
                test_output_belt = test_sorting_center[test_sorting_center["output_belt"] == output_belt]
            
            train_output_belt["no_of_events_boxcox"], lam = boxcox(train_output_belt["no_of_events"])
            train_output_belt["no_of_events_diff"] = train_output_belt["no_of_events_boxcox"].diff()
            #train_output_belt.dropna(inplace=True)
            train_output_belt["no_of_events_diff"].fillna(0, inplace=True)

            p, d, q = hyperparameters[hyperparameters.iloc[:, 0] == output_belt].iloc[0, 1:4]

            arima_model = ARIMA(train_output_belt["no_of_events"], order=(p, d, q)).fit()

            # Forecast for the planning horizon
            forecasts = arima_model.forecast(steps=planning_horizon)

            '''arima_model = ARIMA(train_output_belt["no_of_events_boxcox"], order=(p, d, q)).fit()

            boxcox_forecast = arima_model.forecast(planning_horizon)

            def safe_inv_boxcox(y_series, lambda_value):
                # Ensure the input is a Pandas Series or DataFrame column
                if not isinstance(y_series, (pd.Series, pd.DataFrame)):
                    raise ValueError("Input must be a Pandas Series or DataFrame column.")
                
                # Apply the inverse Box-Cox transformation safely
                def transform_value(y):
                    base = lambda_value * y + 1
                    return base ** (1 / lambda_value) if base > 0 else np.nan
                
                # Apply the transformation element-wise
                return y_series.apply(transform_value)

            # Example usage
            forecasts = safe_inv_boxcox(boxcox_forecast, lam)

            if round(lam,4) == -0.2721:
                print("Hoi")
            #forecasts = inv_boxcox(boxcox_forecast, lam)'''

            if sorting_center_name == "VANTAA" and output_belt in [109]:
                plot_forecast(starting_date, planning_horizon, train_output_belt, forecasts, "ARIMA", test_output_belt)

            if test_df.shape[0] == 0: 
                if sorting_center_name not in forecast_dict:
                    forecast_dict[sorting_center_name] = {}
                
                forecast_dict[sorting_center_name][output_belt] = forecasts
                
            elif test_df.shape[0] > 0: 
                for day in range(planning_horizon):
                    actual = test_output_belt.iloc[day]["no_of_events"]
                    forecast = forecasts.iloc[day]

                    squared_difference = (actual - forecast) ** 2
                    absolute_difference = abs(actual - forecast)

                    if math.isnan(squared_difference):
                        print("The number is NaN")

                    if sorting_center_name not in daily_errors:
                        daily_errors[sorting_center_name] = {}
                    if day not in daily_errors[sorting_center_name]:
                        daily_errors[sorting_center_name][day] = {}
                        daily_errors[sorting_center_name][day]["mse"] = []
                        daily_errors[sorting_center_name][day]["mae"] = []
                    
                    daily_errors[sorting_center_name][day]["mse"].append(squared_difference)
                    daily_errors[sorting_center_name][day]["mae"].append(absolute_difference)

    if test_df.shape[0] > 0: 
        for sorting_center_name in sorting_center_names:
            mse = {}
            mae = {}
            for day in range(planning_horizon):
                mse[day] = sum(daily_errors[sorting_center_name][day]["mse"]) / len(daily_errors[sorting_center_name][day]["mse"])
                mae[day] = sum(daily_errors[sorting_center_name][day]["mae"]) / len(daily_errors[sorting_center_name][day]["mae"])

            MSE_dict[sorting_center_name] = sum(mse.values()) / len(mse)
            VSE_dict[sorting_center_name] = np.var(list(mse.values()), ddof=1)
            MAE_dict[sorting_center_name] = sum(mae.values()) / len(mae)

        daily_mse = {}
        daily_mae = {}

        for day in range(planning_horizon):
            mse = 0
            mae = 0
            n_output_belts = 0
            for sorting_center_name in sorting_center_names:
                mse += sum(daily_errors[sorting_center_name][day]["mse"])
                mae += sum(daily_errors[sorting_center_name][day]["mae"])
                n_output_belts += len(daily_errors[sorting_center_name][day]["mse"])
            daily_mse[day] = mse / n_output_belts
            daily_mae[day] = mae / n_output_belts

        MSE_dict["total"] = sum(daily_mse.values()) / len(daily_mse)
        VSE_dict["total"] = np.var(list(daily_mse.values()), ddof=1)
        MAE_dict["total"] = sum(daily_mae.values()) / len(daily_mae)

        with open("Results/results_ARIMA.csv", mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Sorting center', 'MSE', 'VSE', 'MAE'])
            
            for key in MSE_dict.keys():
                writer.writerow([key, MSE_dict[key], VSE_dict[key], MAE_dict[key]])

    return forecast_dict

### Predictions ARIMA model
Make forecasts for all sorting centers and report the important KPIs to evaluate the performance. These are the mean squared error (MSE), variance squared error (VSE) and mean absolute error (MAE)

In [ ]:
planning_horizon = 14

train = df[df["month"] <= 9] # Train on first 9 months
test = df[(df["month"] == 10) & (df["day"] <= planning_horizon)] # Test on first two weeks of 10th month

starting_date = train.iloc[0]["scanning_date"]

start = datetime.now()
forecast_ARIMA(starting_date, planning_horizon, train, test)
end = datetime.now()

print("Running time predicting ARIMA is: " + str(end-start))

In [ ]:
sorting_center_names = df["sorting_center_name"].unique()
MSE_dict = {}
VSE_dict = {}
MAE_dict = {}
daily_errors = {}

for sorting_center_name in sorting_center_names:
    hyperparameters_df = pd.read_csv("Data/hyperparameters ARIMA/hyperparameters_ARIMA_{}.csv".format(sorting_center_name))
    hyperparameterList = [tuple(row) for row in hyperparameters_df.to_numpy()]
    daily_errors_sorting_center, mse, mae = predict_ARIMA(df, sorting_center_name, hyperparameterList)

    daily_errors[sorting_center_name] = daily_errors_sorting_center
    MSE_dict[sorting_center_name] = sum(mse.values()) / len(mse)
    VSE_dict[sorting_center_name] = np.var(list(mse.values()), ddof=1)
    MAE_dict[sorting_center_name] = sum(mae.values()) / len(mae)
    
daily_mse = {}
daily_mae = {}

for day in range(len(daily_errors)):
    mse = 0
    mae = 0
    n_output_belts = 0
    for sorting_center_name in sorting_center_names:
        mse += sum(daily_errors[sorting_center_name][day]["mse"])
        mae += sum(daily_errors[sorting_center_name][day]["mae"])
        n_output_belts += len(daily_errors[sorting_center_name][day]["mse"])
    daily_mse[day] = mse / n_output_belts
    daily_mae[day] = mae / n_output_belts

MSE_dict["total"] = sum(daily_mse.values()) / len(daily_mse)
VSE_dict["total"] = np.var(list(daily_mse.values()), ddof=1)
MAE_dict["total"] = sum(daily_mae.values()) / len(daily_mae)

with open("Results/results_ARIMA.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Sorting center', 'MSE', 'VSE', 'MAE'])
    
    for key in MSE_dict.keys():
        writer.writerow([key, MSE_dict[key], VSE_dict[key], MAE_dict[key]])


## LSTM/GRU

## Conclusion
Write about pro and cons

In [ ]:
# Conclusion

## Dashboard forecast (Tom)
Standardize input + output
Make accessible for forecast

In [ ]:
# Make the training functions for all models such that the training can be saved and used for prediction every time when needed

def train_model(data, model):
    data = prepare_data(data) # Ensure that data is ready for training

    if model == "Linear Regression":
        #train_LR(data)
        print("Training is finished")
    elif model == "ARIMA":
        train_ARIMA(data)
        print("Training is finished")
    elif model == "Neural Network":
        #train_NN(data)
        print("Training is finished")
    '''elif model == "LSTM":
        #train_LSTM(data)
        print("Training is finished")'''

In [ ]:
def make_forecast(start_date, planning_horizon, data, model):
    if model == "Linear Regression":
        #forecast = forecast_LR(start_date, planning_horizon)             # If data is needed for forecast, then add to the function
        print("Forecast finished")
    elif model == "ARIMA":
        forecast = forecast_ARIMA(start_date, planning_horizon, data)
        print("Forecast finished")
    elif model == "Neural Network":
        #forecast = forecast_NN(start_date, planning_horizon)             # If data is needed for forecast, then add to the function
        print("Forecast finished")
    '''elif model == "LSTM":
        #forecast = forecast_LSTM(start_date, planning_horizon)           # If data is needed for forecast, then add to the function
        print("Forecast finished")'''
        
    return forecast

In [ ]:
def save_forecast(start_date, planning_horizon, model, forecast):
    for sorting_center_name, output_belts in forecast.items():
        # Define the file name
        file_name = f"Results/forecast_{model}_{sorting_center_name}.csv"
        
        # Open the file for writing
        with open(file_name, mode='w', newline='') as file:
            writer = csv.writer(file)
            
            # Construct the list of dates for the forecast period
            dates = [(start_date + timedelta(days=day)).strftime("%Y-%m-%d") for day in range(planning_horizon)]
            
            # Write the header row (Dates)
            header = ['Output Belt'] + dates
            writer.writerow(header)
            
            # Write the forecast data
            for output_belt, forecasts in output_belts.items():
                row = [output_belt] + forecasts.tolist()  # Convert forecasts to list if it's a NumPy array
                writer.writerow(row)
    
    print("Forecasts successfully written to CSV files.")


In [ ]:
def main(start_date, planning_horizon, data, model, train_indicator = False):
    if train_indicator == True:
        train_model(data, model)

    forecast = make_forecast(start_date, planning_horizon, data, model)

    save_forecast(start_date, planning_horizon, forecast)
    #plot_forecast(start_date, planning_horizon, data, prediction, model)   # Currently written to plot one output belt, so process it in the prediction function of your model

In [ ]:
# Main
directory_path = os.getcwd() + "\\Data\\sorting_event_volumes_2023.csv"
df = pd.read_csv(directory_path)

# Input
start_date = "01-10-2023"       # When does the planning horizon start
planning_horizon = 14           # How long is the planning horizon
data = df[df["month"] <= 9]     # Always needed since some models need it for prediction
model = "ARIMA"                 # Which model do you want to use, options are Linear Regression, ARIMA and Neural Network 
train_indicator = False         # Optional, if you want to train the model

main(start_date, planning_horizon, data, model)